# Get recent TCRs from https://vdjdb.cdr3.net/

In [1]:
# !cat SearchTable-2023-07-06-v0.tsv | awk -F "\t" '{OFS="\t"; sub(/.*subject.id\": \"/,"Donor",$15); sub(/\", \"tissue.*}/,"",$15); print $1,$2":"$3,$7,$10,$12,$15}' | awk '$1!=0' | sort -k1,1n -k2,2 | awk '{h1[$1]=h1[$1]"\t"$2; h2[$1]=$4"\t"$6"\t"$3"\t"$5}END{OFS="\t"; for(key in h1) print h1[key],h2[key]}' | sed 's/^\t//' | sort -k4,4 -k3,3 -k1,1 -k2,2 -u > vdjdb20230706.tsv

In [2]:
import pandas as pd
import itertools as it
from tqdm import tqdm

def get_03_data(datapath='vdjdb20230706.tsv'):
    df = pd.read_csv(datapath, sep='\t', header=None)
    df = df[df[0].str.contains(":")]
    df = df[df[1].str.contains(":")]
    df['tcra'] = df[0].str.split(':').apply(lambda x:x[1])
    df['tcrb'] = df[1].str.split(':').apply(lambda x:x[1])
    df = df.drop(columns=[0,1,])
    df = df.rename(columns={2:'peptide', 3:'Donor', 4:'mhc', 5:'EpiSpecies'})
    df['sign'] = 1
    return df


def add_combinations(df):
    df['tcr_combined'] = df['tcra'] + ":" + df['tcrb']
    df['tcr_pep_combined'] = df['tcra'] + ":" + df['tcrb'] + ":" + df['peptide']
    return df

def create_random_combination(original, n=1000, k=10):
    aas = original.tcra.sample(n*k, random_state=n, replace=True).reset_index(drop=True)
    bbs = original.tcrb.sample(n*k, random_state=n+1, replace=True).reset_index(drop=True)
    pps = original.peptide.sample(n*k, random_state=n+2, replace=True).reset_index(drop=True)
    
    neg = pd.concat([aas.to_frame(),bbs.to_frame(),pps.to_frame()], axis=1).drop_duplicates(
        subset=['tcra','tcrb','peptide'], )
    
#     neglist = []
#     ii = 0
#     for a, b, c in tqdm(it.product(aas, bbs, pps)):
#         neglist.append((a, b, c))
#         ii +=1
#         if ii > n*k:
#             break
#     dd = pd.DataFrame(neglist).rename(columns={0:'tcra', 1:'tcrb', 2:'peptide'})
    
    dd = add_combinations(neg)
    dd['sign'] = 0
    return dd

In [3]:
train_allDataTrainedModel = add_combinations(pd.read_parquet("./../data/train_allDataTrainedModel.parquet"))

test_allDataTrainedModel = add_combinations(pd.read_parquet("./../data/valid_allDataTrainedModel.parquet"))

entire_data = pd.concat([train_allDataTrainedModel, test_allDataTrainedModel]).drop_duplicates(
        subset=['tcra','tcrb','peptide'], )

entire_data

,tcra,tcrb,va,ja,vb,jb,t_cell_type,peptide,protein,mhc,sign,tcr_combined,tcr_pep_combined
0,CAVENYGQNFVF,CASSYDSPTEAFF,TRAV22*01,TRAJ26*01,TRBV4-3*01,TRBJ1-1*01,MHCI,GPGHKARVL,Gag,HLA-B*07:02,0,CAVENYGQNFVF:CASSYDSPTEAFF,CAVENYGQNFVF:CASSYDSPTEAFF:GPGHKARVL
1,CAAGGSGGYQKVTF,CASSLADRNTEAFF,TRAV13-1*01,TRAJ13*01,TRBV5-1*01,TRBJ1-1*01,MHCI,AVFDRKSDAK,EBNA4,HLA-A*11,0,CAAGGSGGYQKVTF:CASSLADRNTEAFF,CAAGGSGGYQKVTF:CASSLADRNTEAFF:AVFDRKSDAK
2,CALSELGGATNKLIF,CASSLVGLPSSYNSPLHF,TRAV19*01,TRAJ32*01,TRBV7-8*01,TRBJ1-6*01,MHCI,ELAGIGILTV,MLANA,HLA-A*02,0,CALSELGGATNKLIF:CASSLVGLPSSYNSPLHF,CALSELGGATNKLIF:CASSLVGLPSSYNSPLHF:ELAGIGILTV
3,CAASHTNTGNQFYF,CASSSPGKSSYNEQFF,TRAV13-1*01,TRAJ49*01,TRBV7-8*01,TRBJ2-1*01,MHCI,KLGGALQAK,IE1,HLA-A*03:01,1,CAASHTNTGNQFYF:CASSSPGKSSYNEQFF,CAASHTNTGNQFYF:CASSSPGKSSYNEQFF:KLGGALQAK
5,CLVGDGTGTASKLTF,CASSWGVGSTEAFF,TRAV4*01,TRAJ44*01,TRBV12-4*01,TRBJ1-1*01,MHCI,KLGGALQAK,IE1,HLA-A*03:01,0,CLVGDGTGTASKLTF:CASSWGVGSTEAFF,CLVGDGTGTASKLTF:CASSWGVGSTEAFF:KLGGALQAK
...,...,...,...,...,...,...,...,...,...,...,...,...,...
349371,CAVSDNYAQGLTF,CASSGSSAETLYF,TRAV3N-3:01,TRAJ2-01,TRBV4-01,TRBJ2-3:01,CD8,EPLPQGQLTAY,BZLF,HLA-B*35:01,0,CAVSDNYAQGLTF:CASSGSSAETLYF,CAVSDNYAQGLTF:CASSGSSAETLYF:EPLPQGQLTAY
349439,CAVRDGGATNKLIF,CASSLELNNYGYTF,None,None,None,None,CD4,SSYRRPVGI,PB1,H-2Kb,0,CAVRDGGATNKLIF:CASSLELNNYGYTF,CAVRDGGATNKLIF:CASSLELNNYGYTF:SSYRRPVGI
349541,CAASDYGSSGNKLIF,CTCSAGTGGYNYAEQFF,TRAV1-01,TRAJ3-01,TRBV1-01,TRBJ2-1:01,CD8,WEDLFCDESLSSPEPPSSSE,LTAg,DRB1*04-01,0,CAASDYGSSGNKLIF:CTCSAGTGGYNYAEQFF,CAASDYGSSGNKLIF:CTCSAGTGGYNYAEQFF:WEDLFCDESLSS...
349569,CAARGSNAKLTF,CASSDATKTEVFF,TRAV7-2:02,TRAJ4-01,TRBV13-1:02,TRBJ1-1:01,CD8,SFHSLHLLF,Tax-1,HLA-A*24:02,0,CAARGSNAKLTF:CASSDATKTEVFF,CAARGSNAKLTF:CASSDATKTEVFF:SFHSLHLLF


In [4]:
df = add_combinations(get_03_data("vdjdb20230706.tsv"))
df = df.drop_duplicates(subset=['tcra','tcrb','peptide'], )

In [5]:
entire_data.drop_duplicates(subset=['tcra','tcrb','peptide'], ).shape

(119046, 13)

In [6]:
con = pd.concat([entire_data, df], )

In [7]:
con.drop_duplicates(subset=['tcra','tcrb','peptide'], )

,tcra,tcrb,va,ja,vb,jb,t_cell_type,peptide,protein,mhc,sign,tcr_combined,tcr_pep_combined,Donor,EpiSpecies
0,CAVENYGQNFVF,CASSYDSPTEAFF,TRAV22*01,TRAJ26*01,TRBV4-3*01,TRBJ1-1*01,MHCI,GPGHKARVL,Gag,HLA-B*07:02,0,CAVENYGQNFVF:CASSYDSPTEAFF,CAVENYGQNFVF:CASSYDSPTEAFF:GPGHKARVL,NaN,NaN
1,CAAGGSGGYQKVTF,CASSLADRNTEAFF,TRAV13-1*01,TRAJ13*01,TRBV5-1*01,TRBJ1-1*01,MHCI,AVFDRKSDAK,EBNA4,HLA-A*11,0,CAAGGSGGYQKVTF:CASSLADRNTEAFF,CAAGGSGGYQKVTF:CASSLADRNTEAFF:AVFDRKSDAK,NaN,NaN
2,CALSELGGATNKLIF,CASSLVGLPSSYNSPLHF,TRAV19*01,TRAJ32*01,TRBV7-8*01,TRBJ1-6*01,MHCI,ELAGIGILTV,MLANA,HLA-A*02,0,CALSELGGATNKLIF:CASSLVGLPSSYNSPLHF,CALSELGGATNKLIF:CASSLVGLPSSYNSPLHF:ELAGIGILTV,NaN,NaN
3,CAASHTNTGNQFYF,CASSSPGKSSYNEQFF,TRAV13-1*01,TRAJ49*01,TRBV7-8*01,TRBJ2-1*01,MHCI,KLGGALQAK,IE1,HLA-A*03:01,1,CAASHTNTGNQFYF:CASSSPGKSSYNEQFF,CAASHTNTGNQFYF:CASSSPGKSSYNEQFF:KLGGALQAK,NaN,NaN
5,CLVGDGTGTASKLTF,CASSWGVGSTEAFF,TRAV4*01,TRAJ44*01,TRBV12-4*01,TRBJ1-1*01,MHCI,KLGGALQAK,IE1,HLA-A*03:01,0,CLVGDGTGTASKLTF:CASSWGVGSTEAFF,CLVGDGTGTASKLTF:CASSWGVGSTEAFF:KLGGALQAK,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27215,CAVQAAREYNFNKFYF,CASSQEGDRVTEAFF,NaN,NaN,NaN,NaN,NaN,RLITGRLQSL,NaN,HLA-A*02:01,1,CAVQAAREYNFNKFYF:CASSQEGDRVTEAFF,CAVQAAREYNFNKFYF:CASSQEGDRVTEAFF:RLITGRLQSL,"Donorhealthy,GB18622",SARS-CoV-2
29604,CAVVPMDSNYQLIW,CASSVGGGVTDTQYF,NaN,NaN,NaN,NaN,NaN,MVNTVAGAMK,NaN,HLA-A*68:01,1,CAVVPMDSNYQLIW:CASSVGGGVTDTQYF,CAVVPMDSNYQLIW:CASSVGGGVTDTQYF:MVNTVAGAMK,Donorpt.7,HomoSapiens
29879,CAGNTGTASKLTF,CASSGLAGGPVSGANVLTF,NaN,NaN,NaN,NaN,NaN,ALHGGWTTK,NaN,HLA-A*03:01,1,CAGNTGTASKLTF:CASSGLAGGPVSGANVLTF,CAGNTGTASKLTF:CASSGLAGGPVSGANVLTF:ALHGGWTTK,sapiens,Homo
29881,CLVGGLARDMRF,CSVALGSDTGELFF,NaN,NaN,NaN,NaN,NaN,LQPFPQPELPYGSGGS,NaN,HLA-DQA1*05,1,CLVGGLARDMRF:CSVALGSDTGELFF,CLVGGLARDMRF:CSVALGSDTGELFF:LQPFPQPELPYGSGGS,sapiens,Homo


In [8]:
# "Recent Data Test Set"

df_RecentDataTestSet = df[~df.tcr_pep_combined.isin(entire_data.tcr_pep_combined)]

print(df_RecentDataTestSet.tcra.isin(entire_data.tcra).value_counts(),
df_RecentDataTestSet.tcrb.isin(entire_data.tcrb).value_counts(),
df_RecentDataTestSet.peptide.isin(entire_data.peptide).value_counts())

False    4790
True      770
Name: tcra, dtype: int64 False    5358
True      202
Name: tcrb, dtype: int64 False    4724
True      836
Name: peptide, dtype: int64


In [9]:
assumed_neg = create_random_combination(
    df_RecentDataTestSet, n=len(df_RecentDataTestSet), k=7)

assumed_neg = assumed_neg[~assumed_neg.tcr_pep_combined.isin(
    df_RecentDataTestSet.tcr_pep_combined.unique().tolist()  + 
    entire_data.tcr_pep_combined.unique().tolist() 
)]

In [10]:
df_RecentDataTestSet = pd.concat([df_RecentDataTestSet, 
                                  assumed_neg.head(len(df_RecentDataTestSet)*5)
                                 ])

In [11]:
df_RecentDataTestSet.nunique()

peptide               838
Donor                 149
mhc                    71
EpiSpecies             24
tcra                 4782
tcrb                 5174
sign                    2
tcr_combined        33183
tcr_pep_combined    33360
dtype: int64

In [12]:
df_RecentDataTestSet.sign.value_counts()

0    27800
1     5560
Name: sign, dtype: int64

In [13]:
# df_RecentDataTestSet.to_parquet("../data/recent_data_test.parquet")

In [14]:
# entire_data.to_parquet("../data/entire_data.parquet")

In [17]:
entire_data

,tcra,tcrb,va,ja,vb,jb,t_cell_type,peptide,protein,mhc,sign,tcr_combined,tcr_pep_combined
0,CAVENYGQNFVF,CASSYDSPTEAFF,TRAV22*01,TRAJ26*01,TRBV4-3*01,TRBJ1-1*01,MHCI,GPGHKARVL,Gag,HLA-B*07:02,0,CAVENYGQNFVF:CASSYDSPTEAFF,CAVENYGQNFVF:CASSYDSPTEAFF:GPGHKARVL
1,CAAGGSGGYQKVTF,CASSLADRNTEAFF,TRAV13-1*01,TRAJ13*01,TRBV5-1*01,TRBJ1-1*01,MHCI,AVFDRKSDAK,EBNA4,HLA-A*11,0,CAAGGSGGYQKVTF:CASSLADRNTEAFF,CAAGGSGGYQKVTF:CASSLADRNTEAFF:AVFDRKSDAK
2,CALSELGGATNKLIF,CASSLVGLPSSYNSPLHF,TRAV19*01,TRAJ32*01,TRBV7-8*01,TRBJ1-6*01,MHCI,ELAGIGILTV,MLANA,HLA-A*02,0,CALSELGGATNKLIF:CASSLVGLPSSYNSPLHF,CALSELGGATNKLIF:CASSLVGLPSSYNSPLHF:ELAGIGILTV
3,CAASHTNTGNQFYF,CASSSPGKSSYNEQFF,TRAV13-1*01,TRAJ49*01,TRBV7-8*01,TRBJ2-1*01,MHCI,KLGGALQAK,IE1,HLA-A*03:01,1,CAASHTNTGNQFYF:CASSSPGKSSYNEQFF,CAASHTNTGNQFYF:CASSSPGKSSYNEQFF:KLGGALQAK
5,CLVGDGTGTASKLTF,CASSWGVGSTEAFF,TRAV4*01,TRAJ44*01,TRBV12-4*01,TRBJ1-1*01,MHCI,KLGGALQAK,IE1,HLA-A*03:01,0,CLVGDGTGTASKLTF:CASSWGVGSTEAFF,CLVGDGTGTASKLTF:CASSWGVGSTEAFF:KLGGALQAK
...,...,...,...,...,...,...,...,...,...,...,...,...,...
349371,CAVSDNYAQGLTF,CASSGSSAETLYF,TRAV3N-3:01,TRAJ2-01,TRBV4-01,TRBJ2-3:01,CD8,EPLPQGQLTAY,BZLF,HLA-B*35:01,0,CAVSDNYAQGLTF:CASSGSSAETLYF,CAVSDNYAQGLTF:CASSGSSAETLYF:EPLPQGQLTAY
349439,CAVRDGGATNKLIF,CASSLELNNYGYTF,None,None,None,None,CD4,SSYRRPVGI,PB1,H-2Kb,0,CAVRDGGATNKLIF:CASSLELNNYGYTF,CAVRDGGATNKLIF:CASSLELNNYGYTF:SSYRRPVGI
349541,CAASDYGSSGNKLIF,CTCSAGTGGYNYAEQFF,TRAV1-01,TRAJ3-01,TRBV1-01,TRBJ2-1:01,CD8,WEDLFCDESLSSPEPPSSSE,LTAg,DRB1*04-01,0,CAASDYGSSGNKLIF:CTCSAGTGGYNYAEQFF,CAASDYGSSGNKLIF:CTCSAGTGGYNYAEQFF:WEDLFCDESLSS...
349569,CAARGSNAKLTF,CASSDATKTEVFF,TRAV7-2:02,TRAJ4-01,TRBV13-1:02,TRBJ1-1:01,CD8,SFHSLHLLF,Tax-1,HLA-A*24:02,0,CAARGSNAKLTF:CASSDATKTEVFF,CAARGSNAKLTF:CASSDATKTEVFF:SFHSLHLLF


In [22]:
entire_data.peptide.drop_duplicates().isin(df_RecentDataTestSet.peptide).value_counts()

False    434
True      44
Name: peptide, dtype: int64

In [19]:
df_RecentDataTestSet.tcr_combined.isin(entire_data.tcr_combined).value_counts()

False    33335
True        25
Name: tcr_combined, dtype: int64

In [21]:
df_RecentDataTestSet.peptide.drop_duplicates().isin(entire_data.peptide).value_counts()

False    794
True      44
Name: peptide, dtype: int64